# 🏀 NBA Playoffs Simulator — Simulación & Visualización
**Proyecto:** Simulé los Playoffs NBA miles de veces… y encontré un contender inesperado

**Notebook 04:** El momento del reveal

Este notebook:
1. Carga el modelo XGBoost entrenado y los perfiles de los 16 equipos
2. Corre la **simulación Monte Carlo** (10,000 iteraciones)
3. Genera las **visualizaciones finales** para el video
4. Identifica al **contender inesperado**

Cada simulación juega las 15 series necesarias para coronar un campeón.
El resultado no es UNA predicción, sino una **distribución de probabilidad**.

In [ ]:
# ============================================================
# SETUP: Dependencias
# ============================================================
!pip install xgboost --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import seaborn as sns
import warnings
import os
import pickle
import json
from collections import Counter

from xgboost import XGBClassifier

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('dark_background')

print('✅ Dependencias cargadas')

In [ ]:
# ============================================================
# Montar Drive y cargar todo
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = '/content/drive/MyDrive/nba-playoffs-simulator'
DATA_DIR = f'{PROJECT_DIR}/data'
MODELS_DIR = f'{PROJECT_DIR}/models'
OUTPUTS_DIR = f'{PROJECT_DIR}/outputs'
os.makedirs(OUTPUTS_DIR, exist_ok=True)

# Cargar modelo
with open(f'{MODELS_DIR}/xgb_playoff_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Cargar features
with open(f'{MODELS_DIR}/feature_columns.txt', 'r') as f:
    FEATURE_COLS = [line.strip() for line in f.readlines() if line.strip()]

# Cargar métricas de validación
with open(f'{MODELS_DIR}/validation_metrics.json', 'r') as f:
    val_metrics = json.load(f)

# Cargar perfiles de equipos actuales
df_profiles = pd.read_csv(f'{DATA_DIR}/team_profiles_2026.csv')

print(f'✅ Todo cargado:')
print(f'  → Modelo: XGBoost ({val_metrics["n_features"]} features)')
print(f'  → Validación: Acc={val_metrics["oof_accuracy"]}, Brier={val_metrics["oof_brier_score"]}')
print(f'  → Features: {FEATURE_COLS}')
print(f'  → Equipos: {len(df_profiles)}')

---
## 🏀 Sección 1: Los 16 equipos de playoffs

Separamos por conferencia, asignamos seeds y mostramos el bracket inicial.
Este es el punto de partida de las 10,000 simulaciones.

In [ ]:
# ============================================================
# 1.1 — Separar conferencias y asignar seeds
# ============================================================

# Asegurar que SEED existe
if 'SEED' not in df_profiles.columns:
    df_profiles['SEED'] = df_profiles['PlayoffRank'].astype(int)

east = df_profiles[df_profiles['Conference'] == 'East'].sort_values('SEED').head(8).reset_index(drop=True)
west = df_profiles[df_profiles['Conference'] == 'West'].sort_values('SEED').head(8).reset_index(drop=True)

print('🏀 BRACKET NBA PLAYOFFS 2025-26\n')
print(f'{"WESTERN CONFERENCE":<40} {"EASTERN CONFERENCE"}')
print(f'{"="*38:<40} {"="*38}')

r1_matchups = [(0,7), (3,4), (2,5), (1,6)]  # 1v8, 4v5, 3v6, 2v7

for sa, sb in r1_matchups:
    w = west.iloc[sa]
    wb = west.iloc[sb]
    e = east.iloc[sa]
    eb = east.iloc[sb]

    w_line = f'({int(w["SEED"])}) {w["TEAM_NAME"]:<22} vs ({int(wb["SEED"])}) {wb["TEAM_NAME"]}'
    e_line = f'({int(e["SEED"])}) {e["TEAM_NAME"]:<22} vs ({int(eb["SEED"])}) {eb["TEAM_NAME"]}'
    print(f'{w_line:<40} {e_line}')

print(f'\n📊 Net Ratings:')
for conf_name, conf_df in [('West', west), ('East', east)]:
    print(f'\n  {conf_name}:')
    for _, t in conf_df.iterrows():
        bar = '█' * max(1, int((t['NET_RATING'] + 10) * 1.5))
        print(f'    ({int(t["SEED"])}) {t["TEAM_NAME"]:<28} {t["NET_RATING"]:+.1f}  {bar}')

---
## ⚙️ Sección 2: Motor de Simulación Monte Carlo

Cada simulación:
1. Juega las 8 series de **Primera Ronda** (4 por conferencia)
2. Con los ganadores, juega las 4 series de **Semifinales de Conferencia**
3. Juega las 2 **Finales de Conferencia**
4. Juega las **NBA Finals**

Cada serie es al **mejor de 7**, con ventaja de local (formato 2-2-1-1-1)
para el mejor seed.

Repetimos esto **10,000 veces** y contamos.

In [ ]:
# ============================================================
# 2.1 — Funciones de simulación
# ============================================================

def get_matchup_probability(team_a, team_b, feature_cols, model):
    """
    Calcula P(Team A gana) usando el XGBoost.
    Team A debe ser el mejor seed (favorito).
    """
    row = {}
    for feat in feature_cols:
        base_feat = feat.replace('_diff', '')
        if base_feat in team_a.index and base_feat in team_b.index:
            val_a = team_a[base_feat]
            val_b = team_b[base_feat]
            if pd.notna(val_a) and pd.notna(val_b):
                if base_feat in ['DEF_RATING', 'TM_TOV_PCT']:
                    row[feat] = val_b - val_a
                else:
                    row[feat] = val_a - val_b
            else:
                row[feat] = 0
        elif feat == 'seed_diff':
            seed_a = team_a.get('SEED', team_a.get('PlayoffRank', 4))
            seed_b = team_b.get('SEED', team_b.get('PlayoffRank', 4))
            row[feat] = seed_b - seed_a
        else:
            row[feat] = 0

    X_matchup = pd.DataFrame([row])[feature_cols]
    return model.predict_proba(X_matchup)[0][1]


def simulate_series(prob_a_wins, rng):
    """
    Simula serie Bo7 con home court advantage.
    Formato 2-2-1-1-1: juegos 1,2,5,7 en casa del mejor seed.
    """
    wins_a, wins_b = 0, 0
    home_a_games = {1, 2, 5, 7}
    home_boost = 0.03

    game_num = 0
    while wins_a < 4 and wins_b < 4:
        game_num += 1
        p = prob_a_wins + (home_boost if game_num in home_a_games else -home_boost)
        p = np.clip(p, 0.05, 0.95)
        if rng.random() < p:
            wins_a += 1
        else:
            wins_b += 1

    return wins_a >= 4, wins_a, wins_b


print('✅ Funciones de simulación definidas')

In [ ]:
# ============================================================
# 2.2 — Pre-calcular probabilidades de todos los matchups posibles
# ============================================================
# Esto acelera la simulación: calculamos una vez, usamos 10,000 veces

all_teams = pd.concat([east, west]).reset_index(drop=True)
n_teams = len(all_teams)

prob_matrix = {}

for i in range(n_teams):
    for j in range(n_teams):
        if i == j:
            continue
        team_a = all_teams.iloc[i]
        team_b = all_teams.iloc[j]
        key = (team_a['TEAM_ID'], team_b['TEAM_ID'])
        prob_matrix[key] = get_matchup_probability(
            team_a, team_b, FEATURE_COLS, model
        )

print(f'✅ {len(prob_matrix)} probabilidades de matchup pre-calculadas')

# Mostrar algunas probabilidades interesantes
print(f'\n📊 Matchups destacados (P = prob del favorito):\n')
for sa, sb in r1_matchups:
    for conf_name, conf_df in [('W', west), ('E', east)]:
        a = conf_df.iloc[sa]
        b = conf_df.iloc[sb]
        key = (a['TEAM_ID'], b['TEAM_ID'])
        p = prob_matrix.get(key, 0.5)
        print(f'  [{conf_name}] ({int(a["SEED"])}) {a["TEAM_NAME"]:<24} '
              f'{p:.1%} vs {1-p:.1%} '
              f'({int(b["SEED"])}) {b["TEAM_NAME"]}')

In [ ]:
# ============================================================
# 2.3 — Motor Monte Carlo completo
# ============================================================

N_SIMULATIONS = 10_000
rng = np.random.default_rng(42)

# Tracking detallado
champion_counts = Counter()
finals_counts = Counter()
conf_finals_counts = Counter()
conf_semis_counts = Counter()
finals_matchup_counts = Counter()
r1_upset_counts = Counter()  # Track first round upsets

# Para tracking detallado de cada simulación
sim_champions = []

print(f'🎲 Corriendo {N_SIMULATIONS:,} simulaciones...\n')

for sim in range(N_SIMULATIONS):
    if (sim + 1) % 2500 == 0:
        print(f'  → {sim + 1:,}/{N_SIMULATIONS:,} completadas')

    conf_champions = {}

    for conf_name, conf_df in [('East', east), ('West', west)]:
        teams = [conf_df.iloc[i] for i in range(8)]

        # ===== ROUND 1 =====
        r1_winners = []
        for sa, sb in r1_matchups:
            a, b = teams[sa], teams[sb]
            key = (a['TEAM_ID'], b['TEAM_ID'])
            prob = prob_matrix.get(key, 0.5)
            a_wins, wa, wb = simulate_series(prob, rng)
            winner = a if a_wins else b
            r1_winners.append(winner)

            # Track upsets
            if not a_wins:
                r1_upset_counts[f"{b['TEAM_NAME']} over {a['TEAM_NAME']}"] += 1

        # ===== CONF SEMIS =====
        for w in r1_winners:
            conf_semis_counts[w['TEAM_NAME']] += 1

        r2_winners = []
        for i in range(0, 4, 2):
            a, b = r1_winners[i], r1_winners[i+1]
            seed_a = a.get('SEED', a.get('PlayoffRank', 4))
            seed_b = b.get('SEED', b.get('PlayoffRank', 4))
            if seed_a <= seed_b:
                key = (a['TEAM_ID'], b['TEAM_ID'])
                prob = prob_matrix.get(key, 0.5)
                a_wins, _, _ = simulate_series(prob, rng)
                winner = a if a_wins else b
            else:
                key = (b['TEAM_ID'], a['TEAM_ID'])
                prob = prob_matrix.get(key, 0.5)
                a_wins, _, _ = simulate_series(prob, rng)
                winner = b if a_wins else a
            r2_winners.append(winner)

        # ===== CONF FINALS =====
        for w in r2_winners:
            conf_finals_counts[w['TEAM_NAME']] += 1

        a, b = r2_winners[0], r2_winners[1]
        seed_a = a.get('SEED', a.get('PlayoffRank', 4))
        seed_b = b.get('SEED', b.get('PlayoffRank', 4))
        if seed_a <= seed_b:
            key = (a['TEAM_ID'], b['TEAM_ID'])
            prob = prob_matrix.get(key, 0.5)
            a_wins, _, _ = simulate_series(prob, rng)
            conf_champion = a if a_wins else b
        else:
            key = (b['TEAM_ID'], a['TEAM_ID'])
            prob = prob_matrix.get(key, 0.5)
            a_wins, _, _ = simulate_series(prob, rng)
            conf_champion = b if a_wins else a

        conf_champions[conf_name] = conf_champion

    # ===== NBA FINALS =====
    ec = conf_champions['East']
    wc = conf_champions['West']

    finals_counts[ec['TEAM_NAME']] += 1
    finals_counts[wc['TEAM_NAME']] += 1

    matchup_key = tuple(sorted([ec['TEAM_NAME'], wc['TEAM_NAME']]))
    finals_matchup_counts[matchup_key] += 1

    # Determinar favorito en Finals por mejor Net Rating
    if ec.get('NET_RATING', 0) >= wc.get('NET_RATING', 0):
        key = (ec['TEAM_ID'], wc['TEAM_ID'])
        prob = prob_matrix.get(key, 0.5)
        a_wins, _, _ = simulate_series(prob, rng)
        champion = ec if a_wins else wc
    else:
        key = (wc['TEAM_ID'], ec['TEAM_ID'])
        prob = prob_matrix.get(key, 0.5)
        a_wins, _, _ = simulate_series(prob, rng)
        champion = wc if a_wins else ec

    champion_counts[champion['TEAM_NAME']] += 1
    sim_champions.append(champion['TEAM_NAME'])

print(f'\n✅ {N_SIMULATIONS:,} simulaciones completadas!')

---
## 📊 Sección 3: Resultados de la Simulación

El momento de la verdad. ¿Qué dice la distribución probabilística?

In [ ]:
# ============================================================
# 3.1 — Tabla completa de resultados
# ============================================================

results_data = []
for _, team in all_teams.iterrows():
    name = team['TEAM_NAME']
    conf = team['Conference']
    seed = int(team['SEED'])
    results_data.append({
        'Team': name,
        'Conference': conf,
        'Seed': seed,
        'Champion %': round(champion_counts.get(name, 0) / N_SIMULATIONS * 100, 2),
        'Finals %': round(finals_counts.get(name, 0) / N_SIMULATIONS * 100, 2),
        'Conf Finals %': round(conf_finals_counts.get(name, 0) / N_SIMULATIONS * 100, 2),
        'Conf Semis %': round(conf_semis_counts.get(name, 0) / N_SIMULATIONS * 100, 2),
        'NET_RATING': team['NET_RATING'],
        'W_PCT': team['W_PCT']
    })

df_results = pd.DataFrame(results_data).sort_values('Champion %', ascending=False)
df_results = df_results.reset_index(drop=True)
df_results.index = df_results.index + 1  # Ranking desde 1

print('🏆 DISTRIBUCIÓN DE PROBABILIDAD DE CAMPEONATO\n')
print(f'   {"Equipo":<30} {"Conf":>5} {"Seed":>5} {"Campeón":>9} '
      f'{"Finals":>8} {"Conf F":>8} {"Net Rtg":>8}')
print('   ' + '─' * 80)

for rank, (_, row) in enumerate(df_results.iterrows(), 1):
    bar = '█' * max(1, int(row['Champion %'] * 1.5))
    print(f'{rank:>2}. {row["Team"]:<30} {row["Conference"]:>5} '
          f'{row["Seed"]:>5} {row["Champion %"]:>8.1f}% '
          f'{row["Finals %"]:>7.1f}% {row["Conf Finals %"]:>7.1f}% '
          f'{row["NET_RATING"]:>+7.1f}  {bar}')

In [ ]:
# ============================================================
# 3.2 — Identificar al contender inesperado
# ============================================================

# Un contender inesperado = equipo cuya probabilidad de campeonato
# es significativamente mayor que lo esperado por su seed

# Expectativa base por seed (distribución empírica histórica)
seed_expected = {
    1: 25.0, 2: 18.0, 3: 12.0, 4: 8.0,
    5: 5.0, 6: 3.0, 7: 1.5, 8: 0.5
}
# Ajustar: hay 2 conferencias, así que dividimos
seed_expected_adj = {k: v / 2 for k, v in seed_expected.items()}

df_results['Expected %'] = df_results['Seed'].map(seed_expected_adj)
df_results['Surprise Factor'] = (
    df_results['Champion %'] / df_results['Expected %'].clip(lower=0.1)
)

# Top contenders inesperados: equipos seed 4-8 con probabilidad alta
# o cualquier equipo cuyo Surprise Factor sea > 1.5
surprises = df_results[
    (df_results['Surprise Factor'] > 1.3) & (df_results['Seed'] >= 3)
].sort_values('Surprise Factor', ascending=False)

print('🔥 CONTENDERS INESPERADOS\n')
print('Equipos con probabilidad de campeonato MAYOR a lo esperado por su seed:\n')

if len(surprises) > 0:
    for _, row in surprises.head(5).iterrows():
        print(f'  🚨 {row["Team"]}')
        print(f'     Seed {int(row["Seed"])} ({row["Conference"]}) → '
              f'Prob campeón: {row["Champion %"]:.1f}% '
              f'(esperado ~{row["Expected %"]:.1f}%, '
              f'factor sorpresa: {row["Surprise Factor"]:.1f}x)')
        print()
else:
    print('  No hay sorpresas marcadas → los favoritos dominan')
    print('  Pero veamos quién tiene la mejor ruta...\n')

# También mostrar el favorito principal
top_team = df_results.iloc[0]
print(f'\n👑 FAVORITO #1: {top_team["Team"]}')
print(f'   Probabilidad: {top_team["Champion %"]:.1f}%')
print(f'   Llega a Finals: {top_team["Finals %"]:.1f}% de las veces')

# Insight: ¿domina o comparte?
top_pct = top_team['Champion %']
if top_pct > 25:
    print(f'\n📊 El favorito DOMINA el espacio probabilístico ({top_pct:.0f}%)')
elif top_pct > 15:
    print(f'\n📊 El favorito lidera pero NO domina ({top_pct:.0f}%) — liga competitiva')
else:
    print(f'\n📊 Nadie domina el espacio probabilístico ({top_pct:.0f}%) — liga MUY abierta')

---
## 🎨 Sección 4: Visualizaciones para el Video

Generamos los gráficos finales optimizados para:
- Fondo oscuro (tema dark de Colab / screen recording)
- Legibilidad en formato vertical (TikTok/Reels)
- Impacto visual máximo para el moment del reveal

In [ ]:
# ============================================================
# 4.1 — VISUAL PRINCIPAL: Distribución de probabilidad de campeón
# ============================================================

fig, ax = plt.subplots(figsize=(14, 10))

# Datos ordenados
plot_data = df_results.sort_values('Champion %', ascending=True).copy()

# Colores: destacar top 3 y contender inesperado
top3_names = df_results.head(3)['Team'].values
surprise_names = surprises['Team'].values if len(surprises) > 0 else []

colors = []
for team in plot_data['Team']:
    if team in surprise_names and team not in top3_names:
        colors.append('#FF6D00')  # Naranja para contender inesperado
    elif team == top3_names[0] if len(top3_names) > 0 else '':
        colors.append('#FFD700')  # Oro para #1
    elif team in top3_names:
        colors.append('#64B5F6')  # Azul para top 3
    else:
        colors.append('#546E7A')  # Gris para el resto

bars = ax.barh(range(len(plot_data)), plot_data['Champion %'],
               color=colors, edgecolor='none', height=0.7)

# Labels
ax.set_yticks(range(len(plot_data)))
ax.set_yticklabels([
    f"({int(row['Seed'])}) {row['Team']}"
    for _, row in plot_data.iterrows()
], fontsize=11)

# Porcentaje al final de cada barra
for i, (_, row) in enumerate(plot_data.iterrows()):
    pct = row['Champion %']
    if pct > 0.5:
        ax.text(pct + 0.3, i, f'{pct:.1f}%',
                va='center', fontsize=10, fontweight='bold', color='white')

ax.set_xlabel('Probabilidad de Campeonato (%)', fontsize=13)
ax.set_title('¿Quién gana el campeonato NBA?\n10,000 simulaciones Monte Carlo',
             fontsize=16, fontweight='bold', pad=20)

# Leyenda
legend_elements = [
    mpatches.Patch(color='#FFD700', label='Favorito #1'),
    mpatches.Patch(color='#64B5F6', label='Top contenders'),
]
if len(surprise_names) > 0:
    legend_elements.append(
        mpatches.Patch(color='#FF6D00', label='Contender inesperado')
    )
ax.legend(handles=legend_elements, loc='lower right', fontsize=11)

ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f%%'))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('championship_distribution.png', dpi=200, bbox_inches='tight',
            facecolor='black', edgecolor='none')
plt.show()

print('\n✅ Visual principal guardado: championship_distribution.png')

In [ ]:
# ============================================================
# 4.2 — VISUAL: Finals matchups más probables
# ============================================================

top_finals = finals_matchup_counts.most_common(10)

fig, ax = plt.subplots(figsize=(14, 7))

matchup_labels = []
matchup_pcts = []
for (team_a, team_b), count in top_finals:
    # Abreviar nombres
    label = f'{team_a}\nvs\n{team_b}'
    matchup_labels.append(label)
    matchup_pcts.append(count / N_SIMULATIONS * 100)

bars = ax.bar(range(len(matchup_labels)), matchup_pcts,
              color='#64B5F6', edgecolor='none', width=0.6)
bars[0].set_color('#FFD700')  # Highlight más probable

ax.set_xticks(range(len(matchup_labels)))
ax.set_xticklabels(matchup_labels, fontsize=8, ha='center')

for bar, pct in zip(bars, matchup_pcts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
            f'{pct:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.set_ylabel('Probabilidad (%)', fontsize=12)
ax.set_title('Top 10 Finals más probables\n¿Si tuvieras que apostar un matchup?',
             fontsize=14, fontweight='bold', pad=15)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('finals_matchups.png', dpi=200, bbox_inches='tight',
            facecolor='black', edgecolor='none')
plt.show()

top_matchup = top_finals[0]
print(f'\n🏆 Finals más probable: {top_matchup[0][0]} vs {top_matchup[0][1]}')
print(f'   Ocurre en {top_matchup[1] / N_SIMULATIONS:.1%} de las simulaciones')

In [ ]:
# ============================================================
# 4.3 — VISUAL: Profundidad de carrera (journey map)
# ============================================================

fig, ax = plt.subplots(figsize=(14, 10))

# Ordenar equipos por prob de campeonato
teams_ordered = df_results.sort_values('Champion %', ascending=False)['Team'].values

stages = ['Conf Semis %', 'Conf Finals %', 'Finals %', 'Champion %']
stage_labels = ['Semifinales\nConf.', 'Finales\nConf.', 'NBA\nFinals', 'Campeón']
stage_colors = ['#546E7A', '#64B5F6', '#FFB74D', '#FFD700']

x = np.arange(len(teams_ordered))
width = 0.2

for i, (stage, label, color) in enumerate(zip(stages, stage_labels, stage_colors)):
    values = [df_results[df_results['Team'] == t][stage].values[0]
              for t in teams_ordered]
    bars = ax.bar(x + i * width, values, width, label=label,
                  color=color, edgecolor='none')

ax.set_xticks(x + width * 1.5)
ax.set_xticklabels([t.replace(' ', '\n') for t in teams_ordered],
                    fontsize=8, rotation=0)
ax.set_ylabel('Probabilidad (%)', fontsize=12)
ax.set_title('Profundidad de carrera: ¿Hasta dónde llega cada equipo?',
             fontsize=14, fontweight='bold', pad=15)
ax.legend(fontsize=10, loc='upper right')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('journey_map.png', dpi=200, bbox_inches='tight',
            facecolor='black', edgecolor='none')
plt.show()

In [ ]:
# ============================================================
# 4.4 — VISUAL: Competitividad de la liga (pie chart / donut)
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

for ax, conf_name in zip(axes, ['West', 'East']):
    conf_data = df_results[df_results['Conference'] == conf_name].sort_values(
        'Champion %', ascending=False
    )

    # Donut chart
    sizes = conf_data['Champion %'].values
    labels = [f"({int(r['Seed'])}) {r['Team'].split()[-1]}"
              for _, r in conf_data.iterrows()]

    # Colores: gradiente por probabilidad
    colors_conf = plt.cm.YlOrRd(np.linspace(0.8, 0.2, len(sizes)))

    wedges, texts, autotexts = ax.pie(
        sizes, labels=labels, autopct=lambda p: f'{p:.1f}%' if p > 3 else '',
        colors=colors_conf, startangle=90,
        pctdistance=0.8, textprops={'fontsize': 9}
    )

    # Donut hole
    centre_circle = plt.Circle((0, 0), 0.5, fc='black')
    ax.add_artist(centre_circle)

    # Total prob in center
    total = sizes.sum()
    ax.text(0, 0, f'{total:.0f}%\ntotal', ha='center', va='center',
            fontsize=14, fontweight='bold', color='white')

    ax.set_title(f'{conf_name}ern Conference', fontsize=14, fontweight='bold', pad=15)

plt.suptitle('Distribución de probabilidades por conferencia',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('conference_split.png', dpi=200, bbox_inches='tight',
            facecolor='black', edgecolor='none')
plt.show()

In [ ]:
# ============================================================
# 4.5 — VISUAL: Upsets más frecuentes en primera ronda
# ============================================================

if r1_upset_counts:
    top_upsets = r1_upset_counts.most_common(8)

    fig, ax = plt.subplots(figsize=(12, 6))

    upset_labels = [u[0] for u in top_upsets]
    upset_pcts = [u[1] / N_SIMULATIONS * 100 for u in top_upsets]

    ax.barh(range(len(upset_labels)), upset_pcts,
            color='#FF5252', edgecolor='none', height=0.6)

    ax.set_yticks(range(len(upset_labels)))
    ax.set_yticklabels(upset_labels, fontsize=10)

    for i, pct in enumerate(upset_pcts):
        ax.text(pct + 0.5, i, f'{pct:.1f}%',
                va='center', fontsize=10, fontweight='bold')

    ax.set_xlabel('Frecuencia del upset (%)', fontsize=12)
    ax.set_title('Upsets más probables en Primera Ronda',
                 fontsize=14, fontweight='bold', pad=15)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()
    plt.savefig('upsets_r1.png', dpi=200, bbox_inches='tight',
                facecolor='black', edgecolor='none')
    plt.show()
else:
    print('No se registraron upsets en primera ronda')

---
## 💡 Sección 5: Insights clave para el video

In [ ]:
# ============================================================
# 5.1 — Resumen de insights narrativos
# ============================================================

print('=' * 60)
print('🎬 INSIGHTS PARA EL GUION DEL VIDEO')
print('=' * 60)

# 1. Favorito
top = df_results.iloc[0]
print(f'\n1️⃣ FAVORITO: {top["Team"]}')
print(f'   → {top["Champion %"]:.1f}% de probabilidad de campeonato')
print(f'   → Llega a Finals en {top["Finals %"]:.1f}% de simulaciones')

# 2. Top 3
print(f'\n2️⃣ TOP 3:')
for i, (_, row) in enumerate(df_results.head(3).iterrows(), 1):
    print(f'   {i}. {row["Team"]} — {row["Champion %"]:.1f}%')

# 3. Contender inesperado
print(f'\n3️⃣ CONTENDER INESPERADO:')
if len(surprises) > 0:
    s = surprises.iloc[0]
    print(f'   🚨 {s["Team"]} (Seed {int(s["Seed"])})')
    print(f'   → {s["Champion %"]:.1f}% de probabilidad (esperado ~{s["Expected %"]:.1f}%)')
    print(f'   → Factor sorpresa: {s["Surprise Factor"]:.1f}x')
else:
    # Buscar el equipo seed más bajo con >5% prob
    dark_horses = df_results[df_results['Seed'] >= 4].head(1)
    if len(dark_horses) > 0:
        dh = dark_horses.iloc[0]
        print(f'   🔥 {dh["Team"]} (Seed {int(dh["Seed"])})')
        print(f'   → {dh["Champion %"]:.1f}% pese a ser seed {int(dh["Seed"])}')

# 4. Competitividad
top5_total = df_results.head(5)['Champion %'].sum()
print(f'\n4️⃣ COMPETITIVIDAD:')
print(f'   → Top 5 equipos concentran {top5_total:.1f}% de probabilidad')
print(f'   → Top 1 tiene {top["Champion %"]:.1f}% → '
      f'{"Liga dominada" if top["Champion %"] > 25 else "Liga abierta"}')

# 5. Finals más probable
top_finals_matchup = finals_matchup_counts.most_common(1)[0]
print(f'\n5️⃣ FINALS MÁS PROBABLE:')
print(f'   → {top_finals_matchup[0][0]} vs {top_finals_matchup[0][1]}')
print(f'   → Ocurre en {top_finals_matchup[1] / N_SIMULATIONS:.1%} de simulaciones')

# 6. Upsets
if r1_upset_counts:
    total_r1_upsets = sum(r1_upset_counts.values())
    avg_upsets_per_sim = total_r1_upsets / N_SIMULATIONS
    print(f'\n6️⃣ UPSETS EN PRIMERA RONDA:')
    print(f'   → Promedio de {avg_upsets_per_sim:.1f} upsets por simulación (de 8 series)')
    top_upset = r1_upset_counts.most_common(1)[0]
    print(f'   → Upset más frecuente: {top_upset[0]} ({top_upset[1]/N_SIMULATIONS:.1%})')

---
## 💾 Sección 6: Guardar todo

In [ ]:
# ============================================================
# 6.1 — Guardar resultados y visualizaciones
# ============================================================
import shutil

# Guardar tabla de resultados
df_results.to_csv(f'{DATA_DIR}/simulation_results.csv', index=True)
print(f'✅ simulation_results.csv guardado')

# Guardar todas las imágenes
images = [
    'championship_distribution.png',
    'finals_matchups.png',
    'journey_map.png',
    'conference_split.png',
    'upsets_r1.png'
]

for img in images:
    if os.path.exists(img):
        shutil.copy(img, f'{OUTPUTS_DIR}/{img}')
        print(f'✅ {img} → outputs/')

print(f'\n📁 Archivos en outputs/:')
for f_name in sorted(os.listdir(OUTPUTS_DIR)):
    print(f'  📄 {f_name}')

---
## ✅ Simulación completada

### Activos generados para el video:

| Visual | Archivo | Momento en video |
|---|---|---|
| Distribución de campeón | `championship_distribution.png` | **REVEAL principal** |
| Finals más probables | `finals_matchups.png` | Después del reveal |
| Journey map | `journey_map.png` | Profundidad de análisis |
| Split por conferencia | `conference_split.png` | Contexto West vs East |
| Upsets primera ronda | `upsets_r1.png` | Generar debate |

### Datos para el guion:
- Favorito con probabilidad
- Contender inesperado con factor sorpresa
- Finals más probable
- Nivel de competitividad de la liga
- Upsets más frecuentes

### ➡️ Siguiente paso: Diseñar la arquitectura narrativa del video
Con estos resultados reales, podemos construir el guion, los hooks y el copy. 🎬